In [1]:
import torch
import numpy as np
import os
from datasets import load_dataset
from tqdm import tqdm
from IPython.display import Audio
os.chdir("../")
os.getcwd()
from audiodiffusion import AudioDiffusion
from matplotlib import pyplot as plt
from PIL import Image

/home/lij47/miniconda3/envs/base2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset('teticio/audio-diffusion-breaks-256')
model_id = "teticio/audio-diffusion-breaks-256"
audio_diffusion = AudioDiffusion(model_id=model_id)
mel = audio_diffusion.pipe.mel

unet/diffusion_pytorch_model.safetensors not found
Loading pipeline components...: 100%|██████████| 3/3 [00:00<00:00, 19.76it/s]


In [30]:
def random_window(x):
    return np.random.randn()

def entropy(x):
    _ , counts = np.unique(x, return_counts=True)
    probabilities = counts / x.shape[-1]
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy

def dynamic_range(x):
    return(10 * np.log10(np.max(x) / np.min(x)))
    
def variance(x):
    return np.var(x)

def top_k_windows(arr, window_size, metric, k, sort_max=True):
    values = []
    for i in range(window_size, arr.shape[-1]-window_size, window_size*3):
        window = arr[i:i+window_size]
        values.append((metric(window), i))
    sorted_list = sorted(values, key=lambda x: x[0], reverse=sort_max)
    return sorted_list[:k]

def measure_norm(input_data, inpaints, mask):
    diff = ((input_data - inpaints) * (mask == False))
    norm = np.sum(diff**2, axis=-1) / np.sum(mask[0] == False)
    norm = np.sqrt(norm) 
    norm = np.mean(norm) 
    return norm

In [31]:
audio_length = 130560
audio_length_sec = 5
gap_size = 0.05
window_size = int(audio_length/audio_length_sec * gap_size)
num_inpaints = 1

* Change index to see different examples
* Possible Metrics are random_window, entropy, dynamic_range, and variance
* Change use_top_k to false to use min k

In [32]:
idx = 50
metric = dynamic_range
use_top_k = False
num_gaps = 10
gap_size = 0.02


In [33]:
audio_length = 130560
audio_length_sec = 5
window_size = int(audio_length/audio_length_sec * gap_size)
num_inpaints = 1

train_entropy_norm_list = []

image = dataset['train'][idx]['image']
audio = mel.image_to_audio(image)
mask = np.ones((audio.shape[0]), dtype=bool)
top_k = top_k_windows(audio, window_size, metric, num_gaps)
for tk in top_k:
    mask[tk[1]:tk[1]+window_size] = False
image_mask = mask[::510]
image_mask = image_mask.reshape(1, 1, 256)

masked_audio = audio * mask

inpaint_image = audio_diffusion.generate_spectrogram_and_audio_from_audio(
    raw_audio=masked_audio,
    batch_size=num_inpaints,
    start_step=900,
    step_generator=torch.Generator(device="cuda"))

inpaint_audio = mel.image_to_audio(Image.fromarray(inpaint_image[0]))

print("Original Audio")
display(Audio(audio, rate=22050))
print("Masked Audio")
display(Audio(masked_audio, rate=22050))
print("Inpainted Audio")
display(Audio(inpaint_audio, rate=22050))
print(f"L2 norm of inpaint and original: {measure_norm(np.array(image)[0], inpaint_image, image_mask)}")

Original Audio


Masked Audio


Inpainted Audio


L2 norm of inpaint and original: 9.134518664693534
